In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Model
import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow_addons as tfa
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Input, Conv2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess_input
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import concatenate
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.densenet import DenseNet169


print("TensorFlow Version:", tf.__version__)

In [ ]:
base_dir = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/"

root_dir = "./"
test_dir = base_dir + "test/"
train_dir = base_dir + "train/"
work_dir = root_dir + "dataset/"

if os.path.exists(work_dir):
    remove_tree(work_dir)
    

os.mkdir(work_dir)
copy_tree(train_dir, work_dir)
copy_tree(test_dir, work_dir)
print("Working Directory Contents:", os.listdir(work_dir))

In [ ]:
WORK_DIR = './dataset/'

CLASSES = [ 'NonDemented',
            'VeryMildDemented',
            'MildDemented',
            'ModerateDemented']

IMG_SIZE = 112
IMAGE_SIZE = [112, 112]
DIM = (IMG_SIZE, IMG_SIZE)

In [ ]:
#Performing Image Augmentation to have more data samples

ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)
train_data_gen = work_dr.flow_from_directory(directory=train_dir, target_size=DIM, batch_size=5121, shuffle=False)

In [ ]:
#Retrieving the data from the ImageDataGenerator iterator

train_data, train_labels = train_data_gen.next()

In [ ]:

print(train_data.shape, train_labels.shape)

In [ ]:
sm = SMOTE(random_state=42)

train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)

train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

print(train_data.shape, train_labels.shape)

In [ ]:
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_dataset = test_datagen.flow_from_directory(
    directory= '/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test',
    target_size=(112, 112),
    class_mode='categorical',  # or 'binary' depending on your use case
    batch_size=32,
    shuffle=False  # Ensure no shuffling
)



In [ ]:
print(test_dataset.samples)

In [ ]:
inception_model = InceptionV3(input_shape=(112,112,3), 
                   include_top=False,
                   weights="imagenet")

In [ ]:
for layer in inception_model.layers:
    layer.trainable=False

In [ ]:
model = Sequential()
model.add(inception_model)
model.add(Dropout(0.5))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))       
model.add(Dense(128,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))       
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4,activation='softmax'))

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D

In [ ]:
resnet_model = ResNet50(input_shape=(112, 112, 3), include_top=False, weights="imagenet")

In [ ]:
for layer in resnet_model.layers:
    layer.trainable=False

In [ ]:
# Building Model and adjust the required parameters

model2=Sequential()
model2.add(resnet_model)
model2.add(Dropout(0.5))
model2.add(Flatten())

model2.add(BatchNormalization())
model2.add(Dense(2048,kernel_initializer='he_uniform'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1024,kernel_initializer='he_uniform'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(512,kernel_initializer='he_uniform'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(256,kernel_initializer='he_uniform'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(4,activation='softmax'))

In [ ]:
from tensorflow.keras import Sequential, Input

In [ ]:
input_shape = Input(shape=(112,112,3))

In [ ]:
resnet = model2(input_shape)
inception = model(input_shape)

In [ ]:
from keras.layers import concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
import keras.backend as K

In [ ]:
merge = concatenate([resnet,inception])
hidden1 = Dense(1024, activation='relu')(merge)
output = Dense(4, activation = 'softmax')(hidden1)
ensemble_model = Model(inputs=input_shape, outputs=output)

In [ ]:
#Defining a custom callback function to stop training our model when accuracy goes above 99%

class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('acc') > 0.99:
            print("\nReached accuracy threshold! Terminating training.")
            self.model.stop_training = True
            
my_callback = MyCallback()

#ReduceLROnPlateau to stabilize the training process of the model
rop_callback = ReduceLROnPlateau(monitor="val_loss", patience=3)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:



earlystopping = EarlyStopping(monitor = 'val_auc',
                              mode = 'max' ,
                              patience = 15,
                              verbose = 1)



In [ ]:
METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc'),
           tf.keras.metrics.SensitivityAtSpecificity(0.5, name='sensitivity'),        
           tf.keras.metrics.Precision(name='precision'),
           tfa.metrics.F1Score(num_classes=4)]

CALLBACKS = [my_callback, rop_callback,earlystopping]
    
ensemble_model.compile(optimizer='rmsprop',
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=METRICS)



In [ ]:
#Fit the training data to the model and validate it using the validation data
EPOCHS = 200

history = ensemble_model.fit(train_data, train_labels, validation_data=(val_data, val_labels), callbacks=CALLBACKS, epochs=EPOCHS)

In [ ]:
# Evaluate the model on the test data
test_scores = ensemble_model.evaluate(test_dataset)
print("Test Loss:", test_scores[0])
print("Test Accuracy:", test_scores[1])
# ... and other metrics


In [ ]:
#Plotting the trend of the metrics during training

fig, ax = plt.subplots(1, 3, figsize = (30, 5))
ax = ax.ravel()

for i, metric in enumerate(["acc", "auc", "loss"]):
    ax[i].plot(history.history[metric])
    ax[i].plot(history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("Epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])

In [ ]:
#Predicting the test data

pred_labels = ensemble_model.predict(test_dataset)

In [ ]:
# Number of batches in the test dataset
num_batches = len(test_dataset)

# Initialize an empty list to store the test labels
all_test_labels = []

# Iterate through the test dataset to extract labels
for _ in range(num_batches):
    _, batch_labels = test_dataset.next()
    all_test_labels.append(batch_labels)

# Concatenate the labels from all batches to get the complete test labels
test_labels = np.concatenate(all_test_labels)

In [ ]:
def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

print(classification_report(test_labels, pred_labels, target_names=CLASSES))

In [ ]:
test_labels = test_dataset.classes

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assuming test_labels are integer labels
one_hot_test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=4)

# Get the predicted class labels from the one-hot encoded format
pred_ls = np.argmax(pred_labels, axis=1)
test_ls = np.argmax(one_hot_test_labels, axis=1)

# Compute the confusion matrix
conf_arr = confusion_matrix(test_ls, pred_ls)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels=CLASSES, yticklabels=CLASSES)

plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
print("Balanced Accuracy Score: {} %".format(round(BAS(test_ls, pred_ls) * 100, 2)))
print("Matthew's Correlation Coefficient: {} %".format(round(MCC(test_ls, pred_ls) * 100, 2)))

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
from skimage.io import imread, imshow

In [ ]:
#test for very mild dementia
dic = test_dataset.class_indices
idc = {k: v for v, k in dic.items()}

img = load_img('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented/26 (50).jpg', target_size=(112, 112, 3))
img = img_to_array(img)
img = img / 255
plt.imshow(img)
plt.axis('off')
img = np.expand_dims(img, axis=0)
predictions = ensemble_model.predict(img)
predicted_class_index = np.argmax(predictions)
predicted_probability = round(np.max(predictions) * 100, 2)

print(predicted_probability, '% chances are there that the image is', idc[predicted_class_index])

In [ ]:
dic = test_dataset.class_indices
idc = {k: v for v, k in dic.items()}

img = load_img('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented/26 (24).jpg', target_size=(112, 112, 3))
img = img_to_array(img)
img = img / 255
imshow(img)
plt.axis('off')
img = np.expand_dims(img, axis=0)
predictions = ensemble_model.predict(img)  # Get predicted probabilities for all classes
predicted_class_index = np.argmax(predictions)  # Get the index of the class with the highest probability
predicted_class_name = idc[predicted_class_index]  # Get the class name using the dictionary

probability = round(predictions[0][predicted_class_index] * 100, 2)

print(probability, '% chances are there that the image is', predicted_class_name)


In [ ]:
#test for very mild dementia
dic = test_dataset.class_indices
idc = {k: v for v, k in dic.items()}

img = load_img('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented/26 (69).jpg', target_size=(112, 112, 3))
img = img_to_array(img)
img = img / 255
plt.imshow(img)
plt.axis('off')
img = np.expand_dims(img, axis=0)
predictions = ensemble_model.predict(img)
predicted_class_index = np.argmax(predictions)
predicted_probability = round(np.max(predictions) * 100, 2)

print(predicted_probability, '% chances are there that the image is', idc[predicted_class_index])

In [ ]:
#test for very mild dementia
dic = test_dataset.class_indices
idc = {k: v for v, k in dic.items()}

img = load_img('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented/27.jpg', target_size=(112, 112, 3))
img = img_to_array(img)
img = img / 255
plt.imshow(img)
plt.axis('off')
img = np.expand_dims(img, axis=0)
predictions = ensemble_model.predict(img)
predicted_class_index = np.argmax(predictions)
predicted_probability = round(np.max(predictions) * 100, 2)

print(predicted_probability, '% chances are there that the image is', idc[predicted_class_index])